In [1]:
import cv2
import itertools as it
import numpy as np
import pickle
from utilities import *

In [2]:
with open(r"D:\Dota 2\Heroes\Pickles\keeper_of_the_light.pickle", 'rb') as fin:
    images = []
    try:
        while True:
            images.append(pickle.load(fin))
    except:
        pass

def fn(image):
    # Take the color channel with the lowest value.  This changes the shape of
    # the image from (60, 160, 3) to (60, 160).
    g = (image[:, :, i] for i in range(image.shape[2]))
    image = min(g, key=lambda image: image.sum())
    return image
images = list(map(fn, images))

# Combine the list of two-dimensional tensors into a single three-dimensional tensor.
images = np.stack(images, axis=0)

# There are 13019 valid images.  That number's prime factors are 47, 277.
print(images.shape, *prime_factors(images.shape[0]))

(13019, 15, 200) 47 277


In [3]:
# Determine the desired whiteness by computing the mean of all whiteness.
desired_whiteness = np.mean(images, axis=0)
print(desired_whiteness.shape, desired_whiteness.min(), desired_whiteness.max())
show_and_wait(desired_whiteness)

# Determine the desired factor by scaling the range of standard deviations
# of all whiteness from [min, max] to [1, 0].
std = np.std(images, axis=0)
desired_factor = (std - np.max(std)) / (np.min(std) - np.max(std))
print(desired_factor.shape)
#print(*[[int(f * 10) for f in e] for e in desired_factor], sep='\n')
show_and_wait(desired_factor)

# Apply a threshold to remove the background.
image = (desired_factor * 255).astype(np.uint8)
threshold, image = cv2.threshold(image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_TOZERO)
print(threshold)
desired_factor = image.astype(np.float32) / 255
#print(*[[int(f * 10) for f in e] for e in desired_factor], sep='\n')
show_and_wait(desired_factor)

# Display examples side-by-side.
def fn(desired_whiteness, desired_factor, color):
    background = np.ones(desired_factor.shape, dtype=np.float32) * color
    image = background * (1.0 - desired_factor) + desired_whiteness * desired_factor
    return image
random_indices = sorted(np.random.uniform(0, len(images), [9]).astype(int))
random_indices = [297, 2599, 3473, 4462, 4883, 5394, 5781, 6485, 11840] # These are good.
g = ((fn(desired_whiteness, desired_factor, images[i].min()), images[i]) for i in random_indices)
l = list(it.chain.from_iterable(g))
show_and_wait(np.vstack(l))

cv2.destroyAllWindows()

(15, 200) 0.071155466 0.98201287
(15, 200)
125.0


In [4]:
# Determine a bounding box for the name based on the desired factor.
image = desired_whiteness
show_and_wait(image)
top, bottom, left = 6, 14, 49
K = image[top:bottom, left + 0:left + 6]
E1 = image[top:bottom, left + 6:left + 12]
E2 = image[top:bottom, left + 12:left + 17]
P = image[top:bottom, left + 17:left + 23]
E3 = image[top:bottom, left + 23:left + 28]
R = image[top:bottom, left + 28:left + 34]
O = image[top:bottom, left + 37:left + 44]
F = image[top:bottom, left + 44:left + 49]
T1 = image[top:bottom, left + 52:left + 58]
H1 = image[top:bottom, left + 58:left + 64]
E4 = image[top:bottom, left + 64:left + 70]
L = image[top:bottom, left + 73:left + 78]
I = image[top:bottom, left + 78:left + 81]
G = image[top:bottom, left + 81:left + 88]
H2 = image[top:bottom, left + 88:left + 95]
T2 = image[top:bottom, left + 95:left + 100]
show_and_wait(np.hstack([K, E1, E2, P, E3, R, O, F, T1, H1, E4, L, I, G, H2, T2]))
cv2.destroyAllWindows()